# Code documented with the help of ChatGPT

# First Settings
Configuration of the execution environment, defines constants for states and actions, and randomly selects a keyword and its alternatives from a pre-defined list, thus preparing the 20-question game to start.

In [ ]:
import sys

In [ ]:
import json
import os
import pandas as pd
import random
import string
import torch
sys.path.append("/kaggle/input/llm-20-questions/llm_20_questions")

from  keywords import KEYWORDS_JSON
from os import path
from pathlib import Path
from random import choice
from string import Template
from transformers import T5Tokenizer, T5ForConditionalGeneration
llm_parent_dir = "/kaggle/input/flan-t5/pytorch/large"
device = None
model = None
tokenizer = None
model_initialized = False
ERROR = "ERROR"
DONE = "DONE"
INACTIVE = "INACTIVE"
ACTIVE = "ACTIVE"
TIMEOUT = "TIMEOUT"
GUESS = "guess"
ASK = "ask"
GUESSER = "guesser"
ANSWERER = "guesser"
keywords_list = json.loads(KEYWORDS_JSON)
keyword_cat = random.choice(keywords_list)
category = keyword_cat["category"]
keyword_obj = random.choice(keyword_cat["words"])
keyword = keyword_obj["keyword"]
alts = keyword_obj["alts"]

# Guesser Agent

The <i>guesser_agent</i> function creates dynamic prompts based on the current game state and turn type, and then uses a language model to generate the next action in the game. Specifically:

1. Builds a history of questions and answers so far.
1. Choose the appropriate prompt (question or guess) based on the turn type.
1. Generates the next action by calling a language model with the constructed prompt.

In [ ]:
def guesser_agent(obs):
    info_prompt = """You are playing a game of 20 questions where you ask the questions and try to figure out the keyword, which will be a real or fictional person, place, or thing. \nHere is what you know so far:\n{q_a_thread}"""
    questions_prompt = """Ask one yes or no question."""
    guess_prompt = """Guess the keyword. Only respond with the exact word/phrase. For example, if you think the keyword is [paris], don't respond with [I think the keyword is paris] or [Is the kewyord Paris?]. Respond only with the word [paris]."""
    q_a_thread = ""
    for i in range(0, len(obs.answers)):
        q_a_thread = "{}Q: {} A: {}\n".format(
            q_a_thread,
            obs.questions[i],
            obs.answers[i]
        )
    prompt = ""
    if obs.turnType == ASK:
        prompt = "{}{}".format(
            info_prompt.format(q_a_thread=q_a_thread),
            questions_prompt
        )
    elif obs.turnType == GUESS:
        prompt = "{}{}".format(
            info_prompt.format(q_a_thread=q_a_thread),
            guess_prompt
        )
    else:
        return ""
    
    return call_llm(prompt)

# Answerer Agent
The answerer_agent function:

1. Checks if the turn type is "answer".

1. If so, create a prompt by combining info_prompt and answer_question_prompt, replacing the placeholders with the actual values.

1. Calls the call_llm function with the generated prompt to get the response.

1. If the turn is not "answer", returns an empty string.

1. The agents dictionary maps agent types to the respective roles that define their behavior in the game.

In [ ]:
def answerer_agent(obs):
    info_prompt = """You are a very precise answerer in a game of 20 questions. The keyword that the questioner is trying to guess is [the {category} {keyword}]. """
    answer_question_prompt = """Answer the following question with only yes, no, or if unsure maybe: {question}"""
    if obs.turnType == "answer":
        prompt = "{}{}".format(
            info_prompt.format(category=category,keyword=keyword),
            answer_question_prompt.format(question=obs.questions[-1])
        )
        return call_llm(prompt)
    else: 
        return ""
agents = {GUESSER: guesser_agent, ANSWERER: answerer_agent}

# Guesser Action

### Components of the Function
1. **Initialization of the `guessed` Variable**:
   - The variable `guessed` is initialized as `False` to track whether the keyword has been correctly guessed.
   ```python
   guessed = False
   ```
2. **Checking for No Action**:
   - If `active.action` is empty (no action was taken), the status of `active` is set to `ERROR`.
   ```python
   if not active.action:
       active.status = ERROR
   ```
3. **Handling Question-Asking Turn**:
   - If the observer’s turn type (`turnType`) is `ASK` (ask a question):
     - The action (`active.action`) is treated as a question and is limited to 2000 characters.
     - The question is appended to the lists of questions for both the active and inactive observers.
   ```python
   elif active.observation.turnType == ASK:
       question = active.action[:2000]
       active.observation.questions.append(question)
       inactive.observation.questions.append(question)
   ```
4. **Handling Guessing Turn**:
   - If the observer’s turn type is `GUESS` (make a guess):
     - The action (`active.action`) is treated as a guess and is limited to 100 characters.
     - The guess is appended to the lists of guesses for both the active and inactive observers.
   ```python
   elif active.observation.turnType == GUESS:
       guess = active.action[:100]
       active.observation.guesses.append(guess)
       inactive.observation.guesses.append(guess)
   ```
5. **Checking if the Keyword is Guessed**:
   - If there is an action (`active.action`) and the keyword is correctly guessed (`keyword_guessed(active.action)`):
     - The variable `guessed` is set to `True`.
     - The score is calculated as `20 - int(step / 3)`, where `step` represents the number of steps taken in the game.
     - The `end_game` function is called to end the game with the active and inactive observers, passing the score and final states `DONE`.
   ```python
   if active.action and keyword_guessed(active.action):
       guessed = True
       score = 20 - int(step / 3)
       end_game(active, inactive, score, DONE, DONE)
   ```
6. **Return the `guessed` Variable**:
   - The function returns the `guessed` variable, indicating whether the keyword has been correctly guessed.
   ```python
   return guessed
   ```

In [ ]:
def guesser_action(active, inactive, step):
    guessed = False
    if not active.action:
        active.status = ERROR
    elif active.observation.turnType == ASK:
        question = active.action[:2000]
        active.observation.questions.append(question)
        inactive.observation.questions.append(question)
    elif active.observation.turnType == GUESS:
        guess = active.action[:100]
        active.observation.guesses.append(guess)
        inactive.observation.guesses.append(guess)
    if active.action and keyword_guessed(active.action):
        guessed = True
        score = 20 - int(step / 3)
        end_game(active, inactive, score, DONE, DONE)
    return guessed

# End Game
The `end_game` function finalizes the game by:
- Setting the keyword and category for both participants.
- Assigning rewards to both participants.
- Updating the status of both participants.

This ensures that all participants have the correct end-of-game information and that their states are appropriately updated to reflect the conclusion of the game.

In [ ]:
def end_game(active, inactive, reward, status, inactive_status):
    active.observation.keyword = keyword
    active.observation.category = category
    inactive.observation.keyword = keyword
    inactive.observation.category = category
    active.reward = reward
    inactive.reward = reward
    active.status = status
    inactive.status = inactive_status

# Answerer Action

The `answerer_action` function processes the answerer's response to a question by:
- Setting the keyword and category for the active participant.
- Checking if the response is valid and normalizing it to "yes", "no", or "maybe".
- Ending the game with an error if the response is invalid or empty.
- Appending the response to the answers list of both participants.

This function ensures that the game state is appropriately updated based on the answerer's response and handles any errors or invalid responses by ending the game with an error status.

**Updating Answers**

```python
active.observation.answers.append(response)
inactive.observation.answers.append(response)
```
The normalized `response` is appended to the `answers` list of both the `active` and `inactive` participants. This ensures that both participants have a record of the answerer's response.


In [ ]:
def answerer_action(active, inactive):
    active.observation.keyword = keyword
    active.observation.category = category
    response = active.action
    if not response:
        response = "none"
        end_game(active, inactive, -1, ERROR, DONE)
    elif "yes" in response.lower():
        response = "yes"
    elif "no" in response.lower():
        response = "no"
    else:
        response = "maybe"
        end_game(active, inactive, -1, ERROR, DONE)
    active.observation.answers.append(response)
    inactive.observation.answers.append(response)

# Increment turn
The `increment_turn` function:
1. Ends the game after 60 steps if the keyword has not been guessed.
2. Switches the turn type between "ask" and "guess" to alternate the roles of asking questions and making guesses.
3. Updates the statuses of the active and inactive participants to ensure the correct participant is active for the next turn.

This function ensures that the game progresses smoothly, alternates roles between the participants, and properly handles the end of the game if the keyword is not guessed within the allotted turns.

In [ ]:
def increment_turn(active, inactive, step, guessed):
    if step == 59 and not guessed:
        end_game(active, inactive, -1, DONE, DONE)
    elif active.observation.turnType == "guess":
        active.observation.turnType = "ask"
    elif active.observation.turnType == "ask":
        active.observation.turnType = "guess"
        active.status = INACTIVE
        inactive.status = ACTIVE
    else:
        active.status = INACTIVE
        inactive.status = ACTIVE

# Interpreter

### Purpose of the `interpreter` Function

The `interpreter` function manages the state and actions of a game involving two pairs of agents. Each pair consists of an active and an inactive agent, where one agent is asking questions (the "asker") and the other is guessing (the "guesser"). The function updates the state of the game, determines the actions to be taken by each agent, and handles transitions and end conditions.

### Code Explanation

```python
def interpreter(state, env):
    if env.done:
        return state
```

This line checks if the environment (the game) is done. If it is, the function returns the current state without making any changes.

### Isolating Active and Inactive Agents

```python
    active1 = state[0] if state[0].status == ACTIVE else state[1]
    inactive1 = state[0] if state[0].status == INACTIVE else state[1]
    active2 = state[2] if state[2].status == ACTIVE else state[3]
    inactive2 = state[2] if state[2].status == INACTIVE else state[3]
```

These lines identify which agents are active and inactive for each pair. `state[0]` and `state[1]` are the first pair of agents, while `state[2]` and `state[3]` are the second pair.

### Handling Done Status

```python
    if active1.status == DONE and inactive1.status == DONE:
        active1 = None
        inactive1 = None
    if active2.status == DONE or inactive2.status == DONE:
        active2 = None
        inactive2 = None
    if active1 is None and inactive1 is None and active2 is None and inactive2 is None:
        return state
```

These lines set the agents to `None` if both agents in a pair are done. If all agents are done, the function returns the current state, effectively ending the function.

### Processing Steps and End Conditions

```python
    step = state[0].observation.step
    end_early = (active1 and active1.status) in (TIMEOUT, ERROR) or (active2 and active2.status in (TIMEOUT, ERROR))
    either_guessed = False
```

- `step` stores the current step of the game.
- `end_early` checks if either active agent has a status of `TIMEOUT` or `ERROR`, indicating an early termination condition.
- `either_guessed` is a flag to track if any agent has guessed the keyword correctly.

### Handling Active1 Agent

```python
    if active1 is not None:
        guessed = False
        if active1.observation.role == GUESSER:
            guessed = guesser_action(active1, inactive1, step)
            either_guessed = guessed
        else:
            answerer_action(active1, inactive1)
        if active1.status in (TIMEOUT, ERROR):
            end_game(active1, inactive1, 0, active1.status, DONE)
        elif end_early:
            end_game(active1, inactive1, 0, DONE, DONE)
        else:
            increment_turn(active1, inactive1, step, guessed)
```

- If `active1` is not `None`, the function checks the agent's role.
- If `active1` is the guesser, it calls `guesser_action`.
- If `active1` is the answerer, it calls `answerer_action`.
- It then handles end conditions such as `TIMEOUT` or `ERROR` by calling `end_game`.
- If `end_early` is true, it ends the game.
- Otherwise, it calls `increment_turn` to proceed to the next turn.

### Handling Active2 Agent

```python
    if active2 is not None:
        guessed = False
        if active2.observation.role == GUESSER:
            guessed = guesser_action(active2, inactive2, step)
            either_guessed = either_guessed or guessed
        else:
            answerer_action(active2, inactive2)
        if active2.status in (TIMEOUT, ERROR):
            end_game(active2, inactive2, 0, active2.status, DONE)
        elif end_early:
            end_game(active2, inactive2, 0, DONE, DONE)
        else:
            increment_turn(active2, inactive2, step, guessed)
```

This block is similar to the handling of `active1`, but it operates on `active2` and `inactive2`.

### Returning the State

```python
    return state
```

The function returns the updated state after processing the actions and transitions for both pairs of agents.

### Summary

The `interpreter` function:

1. Checks if the game is done and returns the state if it is.
2. Identifies active and inactive agents in each pair.
3. Handles end conditions and transitions between asking and guessing roles.
4. Calls appropriate functions (`guesser_action`, `answerer_action`, `end_game`, `increment_turn`) based on the current role and status of the agents.
5. Updates and returns the game state.

This function is essential for managing the flow of the game, ensuring that each agent takes the correct action based on its role, and handling various end conditions appropriately.

In [ ]:
def interpreter(state, env):
    if env.done:
        return state
    # Isolate the active and inactive agents.
    active1 = state[0] if state[0].status == ACTIVE else state[1]
    inactive1 = state[0] if state[0].status == INACTIVE else state[1]
    active2 = state[2] if state[2].status == ACTIVE else state[3]
    inactive2 = state[2] if state[2].status == INACTIVE else state[3]
    if active1.status == DONE and inactive1.status == DONE:
        active1 = None
        inactive1 = None
    if active2.status == DONE or inactive2.status == DONE:
        active2 = None
        inactive2 = None
    if active1 is None and inactive1 is None and active2 is None and inactive2 is None:
        return state
    step = state[0].observation.step
    end_early = (active1 and active1.status) in (TIMEOUT, ERROR) or (active2 and active2.status in (TIMEOUT, ERROR))
    either_guessed = False
    if active1 is not None:
        guessed = False
        if active1.observation.role == GUESSER:
            guessed = guesser_action(active1, inactive1, step)
            either_guessed = guessed
        else:
            answerer_action(active1, inactive1)
        if active1.status in (TIMEOUT, ERROR):
            end_game(active1, inactive1, 0, active1.status, DONE)
        elif end_early:
            end_game(active1, inactive1, 0, DONE, DONE)
        else:
            increment_turn(active1, inactive1, step, guessed)
    
    if active2 is not None:
        guessed = False
        if active2.observation.role == GUESSER:
            guessed = guesser_action(active2, inactive2, step)
            either_guessed = either_guessed or guessed
        else:
            answerer_action(active2, inactive2)
        if active2.status in (TIMEOUT, ERROR):
            end_game(active2, inactive2, 0, active2.status, DONE)
        elif end_early:
            end_game(active2, inactive2, 0, DONE, DONE)
        else:
            increment_turn(active2, inactive2, step, guessed)
    
    return state

# Renderer

- **`renderer` Function**:
  - Iterates over the game state.
  - Prints the role, interactions, keyword, and score for each agent.
  - Constructs a transcript for `GUESSER` agents showing their questions, answers, and guesses.
  - Prints blank lines for readability between agents.

- **Additional Code**:
  - Constructs the path to a JSON file.
  - Opens and loads the JSON file into a variable named `specification`.

The `renderer` function helps visualize the current state of the game, making it easier to debug or understand the progress, while the additional code is for loading a game specification from a JSON file.

In [ ]:
def renderer(state, env):
    for s in state:
        print("role: ", s.observation.role)
        if s.observation.role == GUESSER:
            transcript = ""
            for i in range(0, len(s.observation.guesses)):
                transcript = "{}Q: {} A: {}\nG: {}\n".format(
                    transcript, s.observation.questions[i],
                    s.observation.answers[i],
                    s.observation.guesses[i]
                )
            print(transcript)
        print("keyword: ", s.observation.keyword)
        print("score: ", s.reward)
        print("")
        print("")
        print("")
    return ""
jsonpath = path.abspath(path.join("/kaggle/input/llm-20-questions/llm_20_questions", "llm_20_questions.json"))
with open(jsonpath) as f:
    specification = json.load(f)

### `html_renderer` Function

```python
def html_renderer():
    jspath = path.abspath(path.join(path.dirname(__file__), "llm_20_questions.js"))
    with open(jspath) as f:
        return f.read()
```

#### Explanation

1. **Construct JavaScript Path**:
    ```python
    jspath = path.abspath(path.join(path.dirname(__file__), "llm_20_questions.js"))
    ```
    - Uses `path.abspath` to get the absolute path to the `llm_20_questions.js` file.
    - Uses `path.join` to concatenate the directory of the current file (`__file__`) with `llm_20_questions.js`.

2. **Open and Read JavaScript File**:
    ```python
    with open(jspath) as f:
        return f.read()
    ```
    - Opens the JavaScript file located at the path `jspath`.
    - Reads the entire content of the file using `f.read()`.
    - Returns the content as a string.

This function reads a JavaScript file named `llm_20_questions.js` and returns its content as a string. This can be useful for embedding JavaScript into an HTML page dynamically.

### `keyword_guessed` Function

```python
def keyword_guessed(guess: str) -> bool:
    def normalize(s: str) -> str:
        t = str.maketrans("", "", string.punctuation)
        return s.lower().replace("the", "").replace(" ", "").translate(t)
    if normalize(guess) == normalize(keyword):
        return True
    for s in alts:
        if normalize(s) == normalize(guess):
            return True
    return False
```

#### Explanation

1. **Define `normalize` Function**:
    ```python
    def normalize(s: str) -> str:
        t = str.maketrans("", "", string.punctuation)
        return s.lower().replace("the", "").replace(" ", "").translate(t)
    ```
    - `normalize` is a helper function to standardize strings for comparison.
    - Removes all punctuation from the string using `str.maketrans`.
    - Converts the string to lowercase with `s.lower()`.
    - Removes occurrences of "the" and spaces by replacing them with an empty string.
    - Uses `translate(t)` to remove punctuation.

2. **Check If Guess Matches Keyword**:
    ```python
    if normalize(guess) == normalize(keyword):
        return True
    ```
    - Normalizes both `guess` and `keyword`.
    - Compares the normalized strings.
    - If they match, returns `True`.

3. **Check If Guess Matches Any Alternate Keywords**:
    ```python
    for s in alts:
        if normalize(s) == normalize(guess):
            return True
    ```
    - Iterates over each alternate keyword in `alts`.
    - Normalizes and compares each alternate keyword with the normalized `guess`.
    - If any match, returns `True`.

4. **Return False If No Match**:
    ```python
    return False
    ```
    - If neither the keyword nor any alternates match the guess, returns `False`.

The `keyword_guessed` function checks if a given guess matches the target keyword or any of its alternate keywords. It uses the `normalize` function to standardize the strings by removing punctuation, converting to lowercase, and removing spaces and the word "the". This ensures that the comparison is case-insensitive and ignores common variations like punctuation and extra spaces.

In [ ]:
def html_renderer():
    jspath = path.abspath(path.join("/kaggle/input/llm-20-questions/llm_20_questions", "llm_20_questions.js"))
    with open(jspath) as f:
        return f.read()
def keyword_guessed(guess: str) -> bool:
    def normalize(s: str) -> str:
      t = str.maketrans("", "", string.punctuation)
      return s.lower().replace("the", "").replace(" ", "").translate(t)
    if normalize(guess) == normalize(keyword):
      return True
    for s in alts:
      if normalize(s) == normalize(guess):
        return True
    return False

### `call_llm` Function

```python
def call_llm(prompt: str) -> str:
    global model_initialized
    global device
    global model
    global tokenizer
    
    if not model_initialized:
        if os.path.exists(llm_parent_dir) and len(os.listdir(llm_parent_dir)) > 0:
            dirs = os.listdir(llm_parent_dir)
            llm_dir = "{}/{}".format(llm_parent_dir, dirs[0])
            device = "cuda:0" if torch.cuda.is_available() else "cpu"
            model = T5ForConditionalGeneration.from_pretrained(llm_dir).to(device)
            tokenizer = T5Tokenizer.from_pretrained(llm_dir)
            model_initialized = True
        else:
            print("t5-flan model required to use default agents. Add any version of the large model.")
            print("https://www.kaggle.com/models/google/flan-t5/frameworks/pyTorch/variations/large.")
            raise Exception("t5-flan model required to use default agents.")
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return answer[0]
```

#### Explanation

1. **Global Variables**:
    ```python
    global model_initialized
    global device
    global model
    global tokenizer
    ```
    - Declares the use of global variables: `model_initialized`, `device`, `model`, and `tokenizer`.

2. **Check Model Initialization**:
    ```python
    if not model_initialized:
        if os.path.exists(llm_parent_dir) and len(os.listdir(llm_parent_dir)) > 0:
    ```
    - Checks if the model has already been initialized.
    - If not, it checks if the directory `llm_parent_dir` exists and contains files.

3. **Load Model and Tokenizer**:
    ```python
    dirs = os.listdir(llm_parent_dir)
    llm_dir = "{}/{}".format(llm_parent_dir, dirs[0])
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model = T5ForConditionalGeneration.from_pretrained(llm_dir).to(device)
    tokenizer = T5Tokenizer.from_pretrained(llm_dir)
    model_initialized = True
    ```
    - Lists the directories inside `llm_parent_dir`.
    - Constructs the path to the model directory.
    - Sets the device to GPU if available, otherwise uses CPU.
    - Loads the T5 model and tokenizer from the specified directory and moves the model to the selected device.
    - Sets `model_initialized` to `True` to avoid re-initialization in subsequent calls.

4. **Handle Missing Model Directory**:
    ```python
    else:
        print("t5-flan model required to use default agents. Add any version of the large model.")
        print("https://www.kaggle.com/models/google/flan-t5/frameworks/pyTorch/variations/large.")
        raise Exception("t5-flan model required to use default agents.")
    ```
    - If the model directory does not exist or is empty, prints an error message and raises an exception.

5. **Prepare Inputs for Model**:
    ```python
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    ```
    - Tokenizes the input `prompt` and converts it to PyTorch tensors.
    - Moves the tensors to the selected device (CPU or GPU).

6. **Generate Outputs**:
    ```python
    outputs = model.generate(**inputs)
    ```
    - Uses the model to generate outputs from the tokenized inputs.

7. **Decode Outputs**:
    ```python
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return answer[0]
    ```
    - Decodes the generated outputs back into text, skipping special tokens.
    - Returns the first element of the decoded outputs as the final answer.

### Summary

The `call_llm` function is responsible for interacting with a pre-trained T5 model for generating responses based on a given prompt. It ensures that the model and tokenizer are loaded and initialized only once. If the model is not initialized, it loads the model and tokenizer from the specified directory, sets up the device (CPU/GPU), and marks the model as initialized. Then, it tokenizes the input prompt, generates a response using the model, decodes the response, and returns it as a string. If the model files are not found, it raises an exception and provides instructions for adding the required model files.

In [ ]:
def call_llm(prompt: str) -> str:
    global model_initialized
    global device
    global model
    global tokenizer
    if not model_initialized:
        if os.path.exists(llm_parent_dir) and len(os.listdir(llm_parent_dir)) > 0:
            dirs = os.listdir(llm_parent_dir)
            llm_dir = "{}/{}".format(llm_parent_dir, dirs[0])
            device = "cuda:0" if torch.cuda.is_available() else "cpu"
            model = T5ForConditionalGeneration.from_pretrained(llm_dir).to(device)
            tokenizer = T5Tokenizer.from_pretrained(llm_dir)
            model_initialized = True
        else:
            print("t5-flan model required to use default agents. Add any version of the large model.")
            print("https://www.kaggle.com/models/google/flan-t5/frameworks/pyTorch/variations/large.")
            raise Exception("t5-flan model required to use default agents.")
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(prompt)
    return answer[0]

# Run/Debug

*References:* 
1. [test_llm_20_questions.py](https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/llm_20_questions/test_llm_20_questions.py)
2. [Run/Debug LLM 20 Questions in a Notebook](https://www.kaggle.com/code/rturley/run-debug-llm-20-questions-in-a-notebook)

In [ ]:
from kaggle_environments import make
def custom_questioner(obs):
    if obs.turnType == "guess":
        return "banana"
    return "Is it a banana?"
def custom_answerer():
    return "no"
def bad_answerer():
    return "maybe?"
def error_agent():
    raise ValueError
def test_llm_20_q_completes():
    env = make("llm_20_questions", debug=True)
    game_output = env.run([guesser_agent, answerer_agent, guesser_agent, answerer_agent])
    json = env.toJSON()
    env.render(mode="ipython", width=400, height=400)
    assert json["name"] == "llm_20_questions"
    assert json["statuses"] == ["DONE", "DONE", "DONE", "DONE"]
def test_llm_20_q_errors_on_bad_answer():
    env = make("llm_20_questions", debug=True)
    env.run([custom_questioner, custom_answerer, custom_questioner, bad_answerer])
    json = env.toJSON()
    assert json["name"] == "llm_20_questions"
    assert json["rewards"] == [1, 1, 1, None]
    assert json["statuses"] == ["DONE", "DONE", "DONE", "ERROR"]
    print(len(json["steps"]))
    assert len(json["steps"]) == 3
def test_llm_20_q_errors_on_error_answer():
    env = make("llm_20_questions", debug=True)
    env.run([custom_questioner, custom_answerer, custom_questioner, error_agent])
    json = env.toJSON()
    assert json["name"] == "llm_20_questions"
    assert json["rewards"] == [1, 1, 1, None]
    assert json["statuses"] == ["DONE", "DONE", "DONE", "ERROR"]
    print(len(json["steps"]))
    assert len(json["steps"]) == 3
def test_llm_20_q_errors_on_error_question():
    env = make("llm_20_questions", debug=True)
    env.run([custom_questioner, custom_answerer, error_agent, custom_answerer])
    json = env.toJSON()
    assert json["name"] == "llm_20_questions"
    assert json["rewards"] == [1, 1, None, 1]
    assert json["statuses"] == ["DONE", "DONE", "ERROR", "DONE"]
    print(len(json["steps"]))
    assert len(json["steps"]) == 2

In [ ]:
test_llm_20_q_completes()